In [4]:
# @title
# === Cell 1: Setup, Imports, and Environment Configuration (No Changes Needed) ===

# --------------------
# Core Libraries
# --------------------
import os
import re
import glob
import copy
import warnings
import numpy as np
import pandas as pd

# --------------------
# Visualization & Progress Bar
# --------------------
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


# --------------------
# Environment Settings
# --------------------
# Ignore some FutureWarnings from pandas to keep the output clean
warnings.simplefilter(action='ignore', category=FutureWarning)
# Disable SettingWithCopyWarning as we manage our data copies explicitly
pd.options.mode.chained_assignment = None

# 1. Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive successfully mounted.")
except ImportError:
    print("Not in a Google Colab environment, skipping drive mount.")

# 2. Create and change to the working directory
# Using a new, clean directory is recommended
WORK_DIR = '/content/drive/MyDrive/FRL_Slicing_Sim2'
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)
print(f"Current working directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive successfully mounted.
Current working directory: /content/drive/MyDrive/FRL_Slicing_Sim2


In [5]:
# @title
# === Cell 2: Download Dataset and Global Configuration (No Changes Needed) ===

print("\n--- [Initialization] Downloading dataset and setting parameters ---")

# --- 1. Download Dataset ---
DATASET_GIT_URL = "https://github.com/wineslab/colosseum-oran-coloran-dataset.git"
DATASET_LOCAL_NAME = 'colosseum-oran-coloran-dataset'

if not os.path.exists(DATASET_LOCAL_NAME):
    print(f"Downloading dataset from {DATASET_GIT_URL}...")
    # Use -q (quiet) to reduce verbose output
    !git clone -q {DATASET_GIT_URL}
    print("      ✅ Dataset download complete.")
else:
    print(f"      Dataset folder '{DATASET_LOCAL_NAME}' already exists.")

# --- 2. Path Parameters ---
DATASET_DIR = os.path.join(WORK_DIR, DATASET_LOCAL_NAME)
ROME_STATIC_MEDIUM_DIR = os.path.join(DATASET_DIR, 'rome_static_medium')
# Intermediate file paths
BS_PARQUET_PATH = os.path.join(WORK_DIR, 'kpi_traces_bs_intermediate0.parquet')
SLICE_PARQUET_PATH = os.path.join(WORK_DIR, 'kpi_traces_slice_intermediate0.parquet')
# Final output path
FINAL_OUTPUT_PATH = os.path.join(WORK_DIR, 'kpi_traces_final_robust0.parquet')

# --- 3. Business Logic Parameters (from README.md) ---
scheduling_policy_map = {
    'sched0': 'RR', 'sched1': 'WF', 'sched2': 'PF'
}
# Note: training_configs_coloran is not used in this simplified pipeline
# but is kept here for completeness if needed in the future.
print("      ✅ Parameter setup complete.")


--- [Initialization] Downloading dataset and setting parameters ---
      Dataset folder 'colosseum-oran-coloran-dataset' already exists.
      ✅ Parameter setup complete.


In [6]:
# @title
# === Cell 3: [Stage 1] Process Raw Data and Generate Intermediate Files (Revised) ===

print("\n--- [Stage 1] Data Preparation Pipeline Started ---")

# --- 1. File Discovery ---
print("\n[1/3] Discovering all relevant CSV file paths...")
all_slice_level_csv_files = sorted(glob.glob(os.path.join(ROME_STATIC_MEDIUM_DIR, "sched*/tr*/exp*/bs*/slices_bs*/*_metrics.csv")))
all_bs_level_csv_files = sorted(glob.glob(os.path.join(ROME_STATIC_MEDIUM_DIR, "sched*/tr*/exp*/bs*/bs*.csv")))
print(f"      Found {len(all_slice_level_csv_files)} slice-level CSVs and {len(all_bs_level_csv_files)} bs-level CSVs.")

# --- 2. Data Processing and Transformation ---
print("\n[2/3] Processing all CSV files...")
processed_bs_dfs = []
processed_slice_dfs = []

# [修正說明] 建立錯誤日誌，用於收集處理失敗的檔案，而不是沉默地跳過。
failed_files_log = []

# Helper function to parse metadata from file paths
def parse_path_info(file_path):
    try:
        match = re.search(r'/(sched\d+)/(tr\d+)/(exp\d+)/(bs\d+)/', file_path)
        if match:
            sched, tr, exp, bs = match.groups()
            return {
                'Scheduling_Policy_Active': scheduling_policy_map.get(sched, 'Unknown'),
                'Training_Config_ID': tr,
                'exp_id': exp,
                'BS_ID': int(bs.replace('bs', ''))
            }
    except (ValueError, TypeError): return {}
    return {}

# Process BS-Level CSVs
for file in tqdm(all_bs_level_csv_files, desc="Processing BS files"):
    info = parse_path_info(file)
    if not info: continue
    try:
        df = pd.read_csv(file)
        df.rename(columns={'time': 'timestamp'}, inplace=True)
        for k, v in info.items(): df[k] = v
        processed_bs_dfs.append(df)
    except Exception as e:
        # [修正說明] 將錯誤資訊記錄到日誌中。
        failed_files_log.append({'file': file, 'error': str(e)})
        continue

# Process Slice-Level CSVs
for file in tqdm(all_slice_level_csv_files, desc="Processing Slice files"):
    info = parse_path_info(file)
    if not info: continue
    try:
        df = pd.read_csv(file, low_memory=False)
        df.drop(columns=[col for col in df.columns if 'Unnamed:' in str(col)], inplace=True)
        # Rename key columns for clarity and consistency
        df.rename(columns={
            'Timestamp': 'timestamp',
            'slice_id': 'Slice_ID',
            'tx_brate downlink [Mbps]': 'Throughput_DL_Mbps',
            'rx_brate uplink [Mbps]': 'Throughput_UL_Mbps',
            'dl_buffer [bytes]': 'Buffer_Occupancy_DL_bytes'
        }, inplace=True)
        for k, v in info.items(): df[k] = v
        processed_slice_dfs.append(df)
    except Exception as e:
        # [修正說明] 將錯誤資訊記錄到日誌中。
        failed_files_log.append({'file': file, 'error': str(e)})
        continue

# --- 3. Concatenate and Save Intermediate Files ---
print("\n[3/3] Concatenating and saving intermediate Parquet files...")
if processed_bs_dfs:
    df_bs_all = pd.concat(processed_bs_dfs, ignore_index=True)
    df_bs_all.to_parquet(BS_PARQUET_PATH, index=False)
    print(f"      ✅ BS-level intermediate file saved to {BS_PARQUET_PATH} (Shape: {df_bs_all.shape})")
else:
    print("      ⚠️ Failed to process any BS-level files.")

if processed_slice_dfs:
    df_slice_all = pd.concat(processed_slice_dfs, ignore_index=True)
    df_slice_all.to_parquet(SLICE_PARQUET_PATH, index=False)
    print(f"      ✅ Slice-level intermediate file saved to {SLICE_PARQUET_PATH} (Shape: {df_slice_all.shape})")
else:
    print("      ⚠️ Failed to process any Slice-level files.")

# [修正說明] 在 Stage 1 結束時，報告所有處理失敗的檔案。
if failed_files_log:
    print(f"\n--- ⚠️ WARNING: Encountered {len(failed_files_log)} file processing errors ---")
    for log_entry in failed_files_log:
        print(f"  - File: {log_entry['file']}\n    Error: {log_entry['error']}")
else:
    print("\n--- Stage 1 Complete (All files processed successfully) ---")


--- [Stage 1] Data Preparation Pipeline Started ---

[1/3] Discovering all relevant CSV file paths...


KeyboardInterrupt: 

In [7]:
# @title
# === Cell 4: [Stage 2] Final Merge and Storage (Revised) ===

print("\n--- [Stage 2] Final Merge Pipeline Started ---")

try:
    # --- STAGE 1: Load Intermediate Files ---
    print("\n[1/5] Loading intermediate Parquet files generated by Cell 3...")
    df_bs = pd.read_parquet(BS_PARQUET_PATH)
    df_slice = pd.read_parquet(SLICE_PARQUET_PATH)
    print(f"      Load complete. df_bs: {df_bs.shape}, df_slice: {df_slice.shape}")

    # --- STAGE 2: Sanitize & Cast Types for Merging ---
    print("\n[2/5] Executing pre-merge type conversions...")
    by_keys = ['BS_ID', 'exp_id', 'Training_Config_ID', 'Scheduling_Policy_Active']
    on_key = 'timestamp'
    all_sort_keys = by_keys + [on_key]

    for name, df in [('df_bs', df_bs), ('df_slice', df_slice)]:
        print(f"    Processing {name}...")
        original_rows = len(df)

        df[on_key] = pd.to_datetime(df[on_key], unit='ms', errors='coerce')
        if not pd.api.types.is_numeric_dtype(df['exp_id']):
            df['exp_id'] = pd.to_numeric(df['exp_id'].str.replace('exp', ''), errors='coerce')
        if not pd.api.types.is_numeric_dtype(df['Training_Config_ID']):
            df['Training_Config_ID'] = pd.to_numeric(df['Training_Config_ID'].str.replace('tr', ''), errors='coerce')

        # [修正說明] 監控因型別轉換失敗而被丟棄的行數。
        df.dropna(subset=all_sort_keys, inplace=True)
        rows_dropped = original_rows - len(df)
        if rows_dropped > 0:
            print(f"      [INFO] Dropped {rows_dropped} rows from {name} due to parsing errors in key columns.")

        for col in ['BS_ID', 'exp_id', 'Training_Config_ID']:
            df[col] = df[col].astype(int)

    # --- STAGE 3: Sort, Deduplicate, and Index for Performance ---
    print("\n[3/5] Sorting, ensuring key uniqueness, and creating index for fast lookup...")
    for name, df in [('df_bs', df_bs), ('df_slice', df_slice)]:
        # [修正說明] 在刪除重複項之前，先計算並回報重複項的數量。
        num_duplicates = df.duplicated(subset=all_sort_keys).sum()
        if num_duplicates > 0:
            print(f"      [INFO] Found and removed {num_duplicates} duplicate key entries in {name}.")

        df.sort_values(by=all_sort_keys, inplace=True)
        df.drop_duplicates(subset=all_sort_keys, keep='first', inplace=True)

    # Create a multi-index on the right DataFrame for fast group lookups
    df_bs_indexed = df_bs.set_index(by_keys)

    # [修正說明] 將 tolerance 設為可配置變數。基於論文中 100ms 的採集頻率，150ms 是比 250ms 更合理的容忍度。
    MERGE_TOLERANCE = pd.Timedelta('150ms')

    # --- STAGE 4: Perform Merge via GroupBy-Apply (Robust Workaround) ---
    print(f"\n[4/5] Iterating through data groups to perform merge operations (tolerance={MERGE_TOLERANCE})...")
    all_merged_groups = []
    grouped_slice = df_slice.groupby(by_keys, sort=False)

    for group_name, left_group in tqdm(grouped_slice, total=len(grouped_slice), desc="Group-wise merge progress"):
        try:
            right_group = df_bs_indexed.loc[group_name]
            merged_group = pd.merge_asof(
                left=left_group.reset_index(drop=True),
                right=right_group.reset_index(drop=True),
                on='timestamp',
                # [修正說明] CRITICAL FIX: 將 direction 改為 'backward'。
                # 這確保我們只用「過去」的基地台狀態來關聯當前的切片狀態，避免了資料洩漏。
                direction='backward',
                tolerance=MERGE_TOLERANCE,
                suffixes=('', '_bs')
            )
            all_merged_groups.append(merged_group)
        except KeyError:
            # 如果 slice data 的分組在 bs data 中找不到對應，則直接保留 slice data
            all_merged_groups.append(left_group)
            continue

    # --- STAGE 5: Final Step: Concatenate and Save ---
    print("\n[5/5] Concatenating all results and saving...")
    if all_merged_groups:
        final_df = pd.concat(all_merged_groups, ignore_index=True)
        final_df.to_parquet(FINAL_OUTPUT_PATH, index=False)

        print("\n✅✅✅ Data preparation pipeline successfully completed! ✅✅✅")
        print(f"   Final output file located at: {FINAL_OUTPUT_PATH}")
        print(f"   Final Shape: {final_df.shape}")
        display(final_df.head())
    else:
        print("⚠️ Failed to merge any data groups.")

except FileNotFoundError:
    print(f"Error: Could not find intermediate Parquet files ({BS_PARQUET_PATH} or {SLICE_PARQUET_PATH}). Please ensure Cell 3 has been run successfully.")
except Exception as e:
    import traceback
    print(f"\nA fatal error occurred during processing: {e}")
    traceback.print_exc()


--- [Stage 2] Final Merge Pipeline Started ---

[1/5] Loading intermediate Parquet files generated by Cell 3...
      Load complete. df_bs: (6534544, 8), df_slice: (35512393, 35)

[2/5] Executing pre-merge type conversions...
    Processing df_bs...
    Processing df_slice...

[3/5] Sorting, ensuring key uniqueness, and creating index for fast lookup...
      [INFO] Found and removed 7 duplicate key entries in df_bs.
      [INFO] Found and removed 29329159 duplicate key entries in df_slice.

[4/5] Iterating through data groups to perform merge operations (tolerance=0 days 00:00:00.150000)...


Group-wise merge progress:   0%|          | 0/3080 [00:00<?, ?it/s]


[5/5] Concatenating all results and saving...

✅✅✅ Data preparation pipeline successfully completed! ✅✅✅
   Final output file located at: /content/drive/MyDrive/FRL_Slicing_Sim2/kpi_traces_final_robust0.parquet
   Final Shape: (6183234, 38)


,timestamp,num_ues,IMSI,RNTI,slicing_enabled,Slice_ID,slice_prb,power_multiplier,scheduling_policy,dl_mcs,...,dl_ri,ul_n,ul_turbo_iters,Scheduling_Policy_Active,Training_Config_ID,exp_id,BS_ID,nof_ue,dl_brate,ul_brate
0,2021-02-17 21:34:29.447,1,1010123456002,70,1,2,5,1,2,3.17647,...,0,0,1.0,PF,0,1,1,1.0,9536.000000,113376.000000
1,2021-02-17 21:34:29.697,1,1010123456002,70,1,2,5,1,2,0.00000,...,0,0,1.0,PF,0,1,1,1.0,2176.000000,37856.000000
2,2021-02-17 21:34:29.981,2,1010123456002,70,1,2,5,1,2,0.00000,...,0,0,0.0,PF,0,1,1,2.0,0.000000,1166.666626
3,2021-02-17 21:34:30.197,4,1010123456002,70,1,2,5,1,2,0.00000,...,0,0,0.0,PF,0,1,1,4.0,3047.619141,989.000000
4,2021-02-17 21:34:30.447,4,1010123456002,70,1,2,5,1,2,0.00000,...,0,0,0.0,PF,0,1,1,4.0,0.000000,0.000000
